In [ ]:
from stoneforge.io.lasparser import parse_las_file
from dlisio import dlis
import numpy as np

VERSION INFORMATION
Well Information
Log_Parameter
Log_Definition
Drilling_Definition
Drilling_Data | Drilling_Definition
Core_Definition
Core_Data[1] | Core_Definition
Core_Data[2] | Core_Definition
Inclinometry_Definition
Inclinometry_Data | Inclinometry_Definition
Test_Definition
Test_Data | Test_Definition
TOPS_Definition
TOPS_Data | TOPS_Definition
Perforations_Definition
Perforations_Data | Perforations_Definition
Log_Data | Log_Definition


In [ ]:


las3_manager = parse_las_file("../stoneforge/datasets/example_las3.las")

for key, value in las3_manager.items():
    print(key)

In [9]:
def clip_dfheader(header_df, idx = None):
    """get header data from DLIS file in dataframe format."""
    if header_df is not None:
        if idx is not None:
            return header_df.iloc[sorted(idx)]
        else:
            return header_df
    else:
        raise ValueError("Header data is not available.")

In [10]:
def _dataframe_to_dict(df):
    data = {}
    for digital_file, frame_group in df.groupby("Digital File"):
        data[digital_file] = {}
        for frame_name, mnemonics_group in frame_group.groupby("Frame Name"):
            data[digital_file][frame_name] = mnemonics_group["Mnemonics"].tolist()
    return data

In [11]:
from dlisio import dlis
import numpy as np

def _parse_dlis(file_path, data_access):
    extracted_data = {}

    with dlis.load(file_path) as files:
        for f in files:
            logical_file_id = str(f)[12:-1]  # Unique identifier for each logical file
            
            if logical_file_id not in data_access:
                continue  # Skip if the logical file is not in the selection

            extracted_data[logical_file_id] = {}

            simple_frames = {frame.name: frame for frame in f.frames}

            for frame_name, mnemonics in data_access[logical_file_id].items():
                if frame_name not in simple_frames:
                    continue  # Skip if the frame is not found

                frame = simple_frames[frame_name]
                extracted_data[logical_file_id][frame_name] = {}

                for channel in frame.channels:
                    if channel.name in mnemonics:
                        mnemonic = channel.name
                        unit = channel.units
                        values = np.array(channel.curves())  # Convert to numpy array

                        extracted_data[logical_file_id][frame_name][mnemonic] = {
                            'unit': unit,
                            'values': values
                        }

    return extracted_data

In [12]:
import os
import pandas as pd 

def _sanitize_filename(name: str) -> str:
    """Replace unsafe filename characters (like / and \) with underscores."""
    return name.replace("/", "_").replace("\\", "_")

def csv_save(data: dict, output_dir: str = "."):
    """
    Save DLIS-like data structure to CSVs, organized by digital file and frame.

    Parameters
    ----------
    data : dict
        Nested dictionary structured as {frame_name: {digital_file: {mnemonic: {unit, values}}}}.
    output_dir : str
        Base directory where the output folders and files will be created.
    """
    for frame_name, digital_files in data.items():
        sanitized_frame_name = _sanitize_filename(frame_name)

        for digital_file, mnemonics in digital_files.items():
            # Create directory for the digital file
            digital_file_path = os.path.join(output_dir, digital_file)
            os.makedirs(digital_file_path, exist_ok=True)

            # Prepare data and units
            df_data = {}
            units = {}

            for mnemonic, content in mnemonics.items():
                df_data[mnemonic] = content["values"]
                units[mnemonic] = content["unit"]

            # Create DataFrame
            df = pd.DataFrame(df_data)

            # Insert units as second row
            units_row = pd.DataFrame([units])
            df_with_units = pd.concat([units_row, df], ignore_index=True)

            # Save CSV with sanitized frame name
            csv_path = os.path.join(digital_file_path, f"{sanitized_frame_name}.csv")
            df_with_units.to_csv(csv_path, index=False)

In [13]:
from stoneforge.io.dlisio_r import DLISAccess

filepath = r"C:\Users\mario\Desktop\br_dlis\IODP_311-U1325A_rab-proc.dlis"

dlis_manager = DLISAccess(filepath, vis=False)
headers = dlis_manager.dlis_dict_headers
print(headers)

df_headers = dlis_manager._dict_to_dataframe(headers)

c = clip_dfheader(df_headers, idx=[0, 8, 3, 9, 19, 23, 26])
d = _dataframe_to_dict(c)
e = _parse_dlis(filepath, d)
csv_save(e, r"C:\Users\mario\Documents\test")
print(e)

#dlis_manager.csv_save(dlis_manager.dlis_dict_headers, output_dir="C:\\Users\\mario\\Documents\\test")

{'RAB_shallow': {'B77185': {'TDEP': {'unit': 'm', 'dim': '1', 'min': np.float32(-3.26136), 'max': np.float32(358.5361), 'long_name': None}, 'BSIM': {'unit': 'ohm.m', 'dim': '2', 'min': np.float32(0.2657987), 'max': np.float32(57.893593), 'long_name': 'Shallow Button Resistivity Image'}, 'BSAV': {'unit': 'ohm.m', 'dim': '1', 'min': np.float32(0.32535496), 'max': np.float32(14.002237), 'long_name': 'Average Shallow Button Resistivity'}}, 'B77226': {'TDEP': {'unit': '0.1 in', 'dim': '1', 'min': np.float32(-960.0), 'max': np.float32(141120.0), 'long_name': None}, 'DEVI': {'unit': 'deg', 'dim': '1', 'min': np.float32(-639.25), 'max': np.float32(1.4099677), 'long_name': 'Hole Deviation'}, 'HAZI': {'unit': 'deg', 'dim': '1', 'min': np.float32(-639.25), 'max': np.float32(132.75981), 'long_name': 'Hole Azimuth'}, 'RB': {'unit': 'deg', 'dim': '1', 'min': np.float32(-639.25), 'max': np.float32(114.913025), 'long_name': 'Relative Bearing'}, 'P1NO': {'unit': 'deg', 'dim': '1', 'min': np.float32(-63

In [15]:
from stoneforge.io.dlisio_r import DLISAccess

#dlis_manager = DLISAccess("../stoneforge/datasets/DSDP_leg_96_hole_616_96_processed_data.dlis", vis=False)
dlis_manager = DLISAccess(r"C:\Users\mario\Desktop\br_dlis\IODP_311-U1325A_lwd-images-proc.dlis", vis=False)
c = dlis_manager.get_header([0, 8, 3, 9, 19, 23, 26, 31, 32, 42])
#data = dlis_manager.return_data()


IndexError: positional indexers are out-of-bounds

In [4]:
data

{'FDC/CNL/GR_main': {'B59067': {'TDEP': {'unit': 'm',
    'values': array([372.4656   , 372.3132   , 372.1608   , ...,  -4.1146564,
            -4.2670565,  -4.4194565], shape=(2474,), dtype=float32)},
   'RHOB': {'unit': 'g/cm3',
    'values': array([2.0898438, 2.0898438, 2.0898438, ..., 1.1201172, 1.1132812,
           1.109375 ], shape=(2474,), dtype=float32)},
   'GR': {'unit': 'gAPI',
    'values': array([133.25     , 133.25     , 133.25     , ...,   3.765625 ,
             3.7519531,   3.7539062], shape=(2474,), dtype=float32)},
   'CALI': {'unit': 'in',
    'values': array([7.8555613, 7.8555613, 7.8555613, ..., 7.898622 , 7.9232283,
           7.898622 ], shape=(2474,), dtype=float32)},
   'NPHI': {'unit': '%',
    'values': array([ 61.279297,  61.279297,  61.279297, ...,  96.728516, 106.640625,
            99.31641 ], shape=(2474,), dtype=float32)}}},
 'FDC/CNL/GR_repeat': {'B59180': {'TDEP': {'unit': 'm',
    'values': array([264.2616   , 264.1092   , 263.9568   , ...,  -6.553

In [ ]:
import os
import pandas as pd

def _sanitize_filename(name: str) -> str:
    """Replace unsafe filename characters (like / and \) with underscores."""
    return name.replace("/", "_").replace("\\", "_")

def csv_save(data: dict, output_dir: str = "."):
    """
    Save DLIS-like data structure to CSVs, organized by digital file and frame.

    Parameters
    ----------
    data : dict
        Nested dictionary structured as {frame_name: {digital_file: {mnemonic: {unit, values}}}}.
    output_dir : str
        Base directory where the output folders and files will be created.
    """
    for frame_name, digital_files in data.items():
        sanitized_frame_name = _sanitize_filename(frame_name)

        for digital_file, mnemonics in digital_files.items():
            # Create directory for the digital file
            digital_file_path = os.path.join(output_dir, digital_file)
            os.makedirs(digital_file_path, exist_ok=True)

            # Prepare data and units
            df_data = {}
            units = {}

            for mnemonic, content in mnemonics.items():
                df_data[mnemonic] = content["values"]
                units[mnemonic] = content["unit"]

            # Create DataFrame
            df = pd.DataFrame(df_data)

            # Insert units as second row
            units_row = pd.DataFrame([units])
            df_with_units = pd.concat([units_row, df], ignore_index=True)

            # Save CSV with sanitized frame name
            csv_path = os.path.join(digital_file_path, f"{sanitized_frame_name}.csv")
            df_with_units.to_csv(csv_path, index=False)

In [5]:
dlis_manager.csv_save(data, output_dir="C:\\Users\\mario\\Documents\\test")